## 查看defectsj patch

In [ ]:
import json
file='input_base.json'
with open(file, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['buggy_lines'])
        print(data1[i]['patch'])

## codellama infill dataset

In [27]:
import json
file_name='codellama_7b_infill.json'

with open('input_base.json', 'r') as file:
    data = json.load(file)

In [20]:
for key in data['data']:
    # format as "<PRE> {pre} <SUF>{suf} <MID>"
    data['data'][key]['input']="<PRE> "+data['data'][key]['pre']+" <SUF>"+data['data'][key]['post']+ " <MID>"

json.dump(data, open(file_name, 'w'), indent=2)

In [ ]:
with open(file_name, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['input'])

## codellama fillme dataset

In [32]:
import json
file_name='codellama_7b_fillme.json'

with open('input_base.json', 'r') as file:
    data = json.load(file)

In [33]:

for key in data['data']:
    # format as "<PRE> {pre} <SUF>{suf} <MID>"
    data['data'][key]['input']=data['data'][key]['pre']+"<FILL_ME>"+data['data'][key]['post']

json.dump(data, open(file_name, 'w'), indent=2)

In [ ]:
with open(file_name, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['input'])

## codellama perference dataset

In [23]:
## 
import json
file_name='codellama_comparison.json'

with open('input_base.json', 'r') as file:
    data = json.load(file)

In [28]:
prompt="""<|role_start|>system<|role_end|>You are an intelligent programming assistant for JAVA.<|role_start|>human<|role_end|>Choose a correct patch from the following two patches to infill the Java code.\n\nJava code:\npublic static double distance(int[] p1, int[] p2) {\n<INFILL>\n  for (int i = 0; i < p1.length; i++) {\n      final int dp = p1[i] - p2[i];\n      sum += dp * dp;\n  }\n  return Math.sqrt(sum);\n}\n\nPatches:\nA. int sum = 0;\nB. double sum = 0;<|role_start|>bot<|role_end|>The correct patch is B."""

for key in data['data']:
    # format as "<PRE> {pre} <SUF>{suf} <MID>"
    data['data'][key]['input']=prompt+"<|role_start|>human<|role_end|>Choose a correct patch from the following two patches to infill the Java code.\n\nJava code:\n"+data['data'][key]['pre']+"<INFILL>\n"+data['data'][key]['post']+ "\n\nPatches:\nA. "+data['data'][key]['patch'].lstrip()+"B. "+data['data'][key]['buggy_lines'].strip()+'<|role_start|>bot<|role_end|>'

json.dump(data, open(file_name, 'w'), indent=2)

In [ ]:
with open(file_name, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['input'])

## codellama infill with buggy lines dataset

In [2]:
import json
file_name='codellama_infill_with_buggy_lines.json'

with open('input_base.json', 'r') as file:
    data = json.load(file)

In [4]:
def count_leading_spaces(s):
    stripped_string = s.lstrip()
    leading_spaces = len(s) - len(stripped_string)
    return leading_spaces-1

def add_buggy_lines(buggy_line, space_num):
    lst=buggy_line.split('\n')
    commented_lst=[space_num*' '+'// buggy code']
    for s in lst:
        if len(s.strip())>0:
             commented_lst+=[space_num*' '+'// '+s[space_num:]]
    return '\n'.join(commented_lst)

In [5]:
for key in data['data']:
    # format as "<PRE> {pre} <SUF>{suf} <MID>"
    pre, suffix, buggy_line =data['data'][key]['pre'], data['data'][key]['post'], data['data'][key]['buggy_lines']
    space_num = count_leading_spaces(buggy_line)
    buggy_line= add_buggy_lines(buggy_line, space_num)
    data['data'][key]['input']='<PRE> '+pre+buggy_line+' <SUF>'+suffix+' <MID>'

json.dump(data, open(file_name, 'w'), indent=2)

In [ ]:
with open(file_name, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['input'])

## Test

In [35]:
import os

os.environ['TRANSFORMERS_CACHE'] = '/datasets/Large_Language_Models'

from transformers import LlamaForCausalLM, CodeLlamaTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

In [36]:
PROMPT = '''public double getChiSquare() {
    double chiSquare = 0;
    for (int i = 0; i < rows; ++i) {
        final double residual = residuals[i];
<FILL_ME>
    }
    return chiSquare;
}'''
input_ids = tokenizer(PROMPT, return_tensors="pt", padding="max_length", max_length=100, truncation=True)["input_ids"]
input_ids

tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     1, 32007,   970,  3765,   679,  1451,
         29875, 29903,  4718,   580,   426,    13,  1678,  3765, 18558, 29903,
          4718,   353, 29871, 29900, 29936,    13,  1678,   363,   313,   524,
           474,   353, 29871, 29900, 29936,   474,   529,  4206, 29936,  8445,
         29875, 29897,   426,    13,  4706,  2186,  3765, 10995,   950,   353,
         10995, 27101, 29961, 29875,  1385,    13, 32008,    13,  1678,   500,
            13,  1678,   736, 18558, 29903,  4718, 29936,    13, 29913, 32009]])

In [41]:
fill_PROMPT='''<PRE> public double getChiSquare() {
    double chiSquare = 0;
    for (int i = 0; i < rows; ++i) {
        final double residual = residuals[i];
 <SUF>
    }
    return chiSquare;
} <MID>'''
fill_input_ids = tokenizer(fill_PROMPT, return_tensors="pt",  padding="max_length", max_length=100, truncation=True)["input_ids"]
fill_input_ids

tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     1, 32007,   970,  3765,   679,  1451,
         29875, 29903,  4718,   580,   426,    13,  1678,  3765, 18558, 29903,
          4718,   353, 29871, 29900, 29936,    13,  1678,   363,   313,   524,
           474,   353, 29871, 29900, 29936,   474,   529,  4206, 29936,  8445,
         29875, 29897,   426,    13,  4706,  2186,  3765, 10995,   950,   353,
         10995, 27101, 29961, 29875,  1385,    13, 32008,    13,  1678,   500,
            13,  1678,   736, 18558, 29903,  4718, 29936,    13, 29913, 32009]])

In [42]:
tokenizer.decode(input_ids[0], skip_special_tokens = False)

'</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s> <PRE> public double getChiSquare() {\n    double chiSquare = 0;\n    for (int i = 0; i < rows; ++i) {\n        final double residual = residuals[i];\n <SUF>\n    }\n    return chiSquare;\n} <MID>'

In [43]:
tokenizer.decode(fill_input_ids[0], skip_special_tokens = False)

'</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s> <PRE> public double getChiSquare() {\n    double chiSquare = 0;\n    for (int i = 0; i < rows; ++i) {\n        final double residual = residuals[i];\n <SUF>\n    }\n    return chiSquare;\n} <MID>'